<a href="https://colab.research.google.com/github/aquibjaved/Bits_and_Pieces_DL/blob/main/positional_embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>